In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import os
import time
from random import randint

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("download.default_directory=" + os.getcwd() + "/Downloads")


Process Flow
-------------------
Search professors from start_page by name
Click Search
Find the first reseach item in the research section
Determine type of publication
Get Abstract if Abstract exists, if not skip to next
Click citation download link
Select option (1 = RIS, 2 = BibTex, 3 = Plain text)
Select option  (1 = citation 2 = citation and abstract)
Download
Go to most recent downloaded file
Read text from that file
Place citation into variable
If abstract exists for citation place into abstract variable if no abstract found insert "No abstract found."

<h2>To do</h2>

faculty,citation,title,type,abstract,book,journal,date,collabs,publisher,edition,series,volume,issue,chapter,pages,issn,isbn,handle,url



In [4]:
def google_scholar_scraper(record,driver):
    
    # Set driver to go to start_page and search a given name with valid scholar profile    
    google_scholar = "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG="
    driver.get(google_scholar)

    time.sleep(3)
    
    # Whether a profile is found is set to false by default
    profile_found = False
    search_bar = driver.find_element_by_class_name('gs_in_txt')
    search_bar.clear()
    search_bar.send_keys("\"" + record.faculty.strip() + "\"" + " CSUN")
    search_bar.send_keys(Keys.ENTER)
    
    # If there's search results
    if not driver.find_elements_by_xpath('//*[@id="gsc_sa_ccl"]/div/p[1]'):
        # A profile was found
        profile_found = True
    # If there's no search results let's try again
    else:
        # A little wait
        time.sleep(3)
        search_bar = driver.find_element_by_class_name('gs_in_txt')
        search_bar.clear()
        search_bar.send_keys("\"" + record.faculty.strip() + "\"" + " Northridge")
        search_bar.send_keys(Keys.ENTER)
        if not driver.find_elements_by_xpath('//*[@id="gsc_sa_ccl"]/div/p[1]'):
            profile_found = True
            
    # If the profile was found
    if profile_found:
        time.sleep(3)
        view_profile_link = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[1]/div/div/h3/a').get_attribute("href")
        citation_collector(record, view_profile_link)
    else:
        f = open("google_scholar_no_results.csv", "a")
        f.write(record.faculty.strip() + ",")
        f.write(record.college + ",")
        f.write(record.department + ",")
        f.write(record.url + "\n")
        f.close()
        
        

In [5]:
def citation_collector(record, url):
    
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    driver.get(url)
    
    # To get all the links we need to click the Show More multiple times to show the whole links 
    gs_bpf_wrapper = driver.find_element_by_id("gsc_bpf_more")

    while not gs_bpf_wrapper.get_attribute("disabled"):
        gs_bpf_wrapper.click()
        time.sleep(3)

    #pull the first citation for the page
    view_citation_links = driver.find_elements_by_class_name('gsc_a_at')

    
    t = 0

    for link in view_citation_links:

        print(t)
        t+=1
        
        not_his_paper = False

        citation = ""
        title = link.text
        paper_type = ""
        book = ""
        journal = ""
        date = ""
        authors = ""
        description = ""
        publisher = ""
        edition = ""
        series = ""
        volume = ""
        issue = ""
        chapter = ""
        pages = ""
        issn = ""
        isbn = ""
        handle = ""


        #time.sleep(3)
        webdriver.ActionChains(driver).move_to_element(link).click(link).perform()
        #link.click()
        
        if driver.find_elements_by_xpath('//*[@id="gsc_vcd_title"]/a'):
            url = driver.find_element_by_xpath('//*[@id="gsc_vcd_title"]/a').get_attribute("href")
        else:
            url = driver.current_url
        
        
        time.sleep(1)

        citation_table = driver.find_element_by_id("gsc_vcd_table")
        citation_table_data_objects= citation_table.find_elements_by_class_name("gs_scl")

        data_category = citation_table_data_objects[0].find_element_by_tag_name("div")

        if data_category.text == "Authors":

            i = 0
            for data in citation_table_data_objects:

                data_category = citation_table_data_objects[i].find_element_by_tag_name("div")

                citation_title = driver.find_element_by_id('gsc_vcd_title')
                title = citation_title.text

                if data_category.text == "Authors":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    if "|" in data_field.text:
                        authors = data_field.text
                        authors.replace("|", " ")
                    else:
                        authors = data_field.text
                    # what second condition was i going to check here?
                    if not record.faculty.strip() in data_field.text:
                        not_his_paper = True
                elif data_category.text == "Publication date":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    if "|" in data_field.text:
                        date = data_field.text
                        date.replace("|", " ")
                    else:
                        date = data_field.text
                    date = data_field.text
                elif data_category.text == "Publisher":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    if "|" in data_field.text:
                        publisher = data_field.text
                        publisher.replace("|", " ")
                    else:
                        publisher = data_field.text
                    publisher = data_field.text
                elif data_category.text == "Description":
                    # this line of code grabs the description
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    data = data_field.text

                    # here is where we want to go through the description and
                    # replace every instance of "\n" with " "
                    desc = data.split("\n")
                    temp = ""
                    for piece in desc:
                        temp += piece + " "
                    if len(temp.split('|')) > 1: 
                        desc = temp.split('|')
                        description = ""
                        for piece in desc:
                            description += piece + " "
                    else:
                        description = temp
                elif data_category.text == "Journal":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    if "|" in data_field.text:
                        journal = data_field.text
                        journal.replace("|", " ")
                    else:
                        journal = data_field.text
                    journal = data_field.text
                elif data_category.text == "Issue":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    if "|" in data_field.text:
                        issue = data_field.text
                        issue.replace("|", " ")
                    else:
                        issue = data_field.text
                    issue = data_field.text
                elif data_category.text == "Volume":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    if "|" in data_field.text:
                        volume = data_field.text
                        volume.replace("|", " ")
                    else:
                        volume = data_field.text
                    volume = data_field.text
                    volume = data_field.text
                elif data_category.text == "Pages":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    if "|" in data_field.text:
                        pages = data_field.text
                        pages.replace("|", " ")
                    else:
                        pages = data_field.text
                    pages = data_field.text
                i = i + 1
                
            citation_file = open('google_scholar_citations.csv', "a")
            citation_file.write(record.faculty.strip() + "|" + citation + "|" + title + "|" + paper_type + "|")
            citation_file.write(description + "|" + book + "|" + journal + "|" + date + "|")
            citation_file.write(authors + "|" + publisher + "|" + edition + "|" + series + "|")
            citation_file.write(volume + "|" + issue + "|" + chapter + "|" + pages + "|")
            citation_file.write(issn + "|" + isbn + "|" + handle + "|" + url + "\n")
            citation_file.close()

        elif data_category.text == "Inventors":
            pass

        elif not_his_paper:
            print("Error: Publication ", i , " did not get captured by program; new decision tree required.")

        close_table = driver.find_element_by_id("gs_md_cita-d-x")
        webdriver.ActionChains(driver).move_to_element(close_table).click(close_table).perform()
        time.sleep(1)
        
    driver.close()
    

[x] view_content

[x] field_items

[x] catalog_content

[x] accordion

In [6]:
view_content = pd.read_csv("catalog_content.csv")
view_content.head()
print(view_content.columns)
view_content.head()

Index(['faculty', 'college', 'department', 'url'], dtype='object')


,faculty,college,department,url
0,Rick Covington,Engineering & Computer Science,Department of Computer Science,https://www.csun.edu/engineering-computer-sci...
1,Prasanta Barkataki,Engineering & Computer Science,Department of Computer Science,https://www.csun.edu/engineering-computer-sci...
2,Mike Barnes,Engineering & Computer Science,Department of Computer Science,https://www.csun.edu/engineering-computer-sci...
3,Kyle Dewey,Engineering & Computer Science,Department of Computer Science,https://www.csun.edu/engineering-computer-sci...
4,Steven Fitzgerald,Engineering & Computer Science,Department of Computer Science,https://www.csun.edu/engineering-computer-sci...


In [7]:
# Check this one to see if any citations were missed
view_content.iloc[77]

faculty                                            Hamid Johari
college                          Engineering & Computer Science
department                 Department of Mechanical Engineering
url            https://www.csun.edu/engineering-computer-sci...
Name: 77, dtype: object

In [16]:
try:
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    for i in range(860,view_content.shape[0]):
        print(i,":",view_content.iloc[i].faculty)
        google_scholar_scraper(view_content.iloc[i],driver)
finally:
    driver.close()

860 : Candida Fernandez-Ghoneim
861 : Ronald Frydman
862 : Joel S Gershon
863 : Hawa Ghaus-Kelley
864 : Randall Gilpin
865 : Daniel Hart
866 : Maria Hirsch
867 : Alison Hunsaker
868 : Tomer Kleinman
869 : Ronald M Klemp
870 : Irv Kodimer
871 : Sandra Krist
872 : Donald J Krotser
873 : Carol Lache
874 : Douglas Lasken
875 : Kerry Youngae Lee
876 : Margaret A Lobb
877 : Pamela Loew
878 : Nancy Madrid
879 : Angela Marino
880 : Leba K Marquez
881 : John McQuilkin
882 : Stephen J Miller
883 : Linda Minster
884 : Nicolas Mize
885 : Linda Mouradian
886 : Carlos Olivares
887 : Larry A Oviatt
888 : Marian P Pasternack
889 : Paul M Payne
890 : Joyce E Petersons
891 : Jeena Philip
892 : Jane S Prochazka
893 : Erica Reyes
894 : Donald F Savarese
895 : Pamela Scherban-Sierra
896 : Gary Schur
897 : Maureen M Sloan
898 : Susan Stivers
899 : Michael Sullivan
900 : James R Szabo
901 : Fataneh Tabatabai
902 : Terri Turner
903 : Judith Vanderbok
904 : Penelope Venola
905 : Grace A Warren
906 : Brenda N W